In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [3]:
df = pd.read_csv('data/amsterdam/amsterdam_full.csv')

In [5]:
import pandas as pd
from bs4 import BeautifulSoup

# Assuming your DataFrame is named 'df' and the HTML content is stored in the 'content' column
# Create an empty DataFrame to store the extracted data
extracted_rows = []

# Initialize the previous header variables
previous_headers = [None] * 6

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Get the URL and HTML content from the current row
    url = row['URL']
    if row['Content'] is not None:
        html_content = row['Content']
    else: 
        continue
    
    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(str(html_content), 'html.parser')
    
    # Find all the headers (h1 to h6) in the HTML
    headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    
    # Iterate over the headers and extract the corresponding text
    for header in headers:
        # Get the header level
        header_level = int(header.name[1])
        
        # Update the previous header variables
        previous_headers[header_level-1] = header.text.strip()
        previous_headers[header_level:] = [None] * (6 - header_level)
        
        # Find the next header of the same level
        next_header = header.find_next_sibling(['h{}'.format(header_level)])
        
        # Find the text under the current header until the next header is encountered
        text = ''
        next_element = header.find_next_sibling()
        while next_element and next_element != next_header:
            text += next_element.text.strip() + ' '
            next_element = next_element.find_next_sibling()
        
        # Append the extracted data as a dictionary to the list
        extracted_rows.append({
            'url': url,
            'h1': previous_headers[0],
            'h2': previous_headers[1],
            'h3': previous_headers[2],
            'h4': previous_headers[3],
            'h5': previous_headers[4],
            'h6': previous_headers[5],
            'text': text.strip()
        })

# Create a new DataFrame using the extracted data
extracted_df = pd.DataFrame(extracted_rows, columns=['url', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'text']) # 2 min


/Users/natalipeeva/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
extracted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121053 entries, 0 to 121052
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     121053 non-null  object
 1   h1      121053 non-null  object
 2   h2      82440 non-null   object
 3   h3      46845 non-null   object
 4   h4      5405 non-null    object
 5   h5      0 non-null       object
 6   h6      0 non-null       object
 7   text    121053 non-null  object
dtypes: object(8)
memory usage: 7.4+ MB


In [7]:
# Tokenize the 'text' column
extracted_df['tokens'] = extracted_df['text'].apply(nltk.word_tokenize)

# Calculate the average number of tokens
average_tokens = extracted_df['tokens'].apply(len).median()

# Print the average number of tokens
print("Average number of tokens in 'text' column:", average_tokens) # 1m

Average number of tokens in 'text' column: 36.0


In [8]:
mean_tokens = extracted_df['tokens'].apply(len).mean()

In [9]:
mean_tokens

75.8752282058272

In [10]:
# Tokenize the 'text' column
extracted_df['tokens'] = extracted_df['text'].apply(nltk.word_tokenize)

# Calculate the statistics
statistics = extracted_df['tokens'].apply(len).describe()

# Extract the desired statistics
median_tokens = statistics['50%']
min_tokens = statistics['min']
max_tokens = statistics['max']

# Print the statistics
print("Median number of tokens in 'text' column:", median_tokens)
print("Minimum number of tokens in 'text' column:", min_tokens)
print("Maximum number of tokens in 'text' column:", max_tokens)


Median number of tokens in 'text' column: 36.0
Minimum number of tokens in 'text' column: 0.0
Maximum number of tokens in 'text' column: 8959.0


In [11]:
# Assuming you have already tokenized the 'text' column and stored it in the 'tokens' column

# Calculate the IQR and define the threshold for outliers
Q1 = extracted_df['tokens'].apply(len).quantile(0.25)
Q3 = extracted_df['tokens'].apply(len).quantile(0.75)
IQR = Q3 - Q1
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR

# Count the number of outliers
outliers_count = ((extracted_df['tokens'].apply(len) < lower_threshold) |
                  (extracted_df['tokens'].apply(len) > upper_threshold)).sum()

# Print the number of outliers
print("Number of outliers according to IQR:", outliers_count)


Number of outliers according to IQR: 10457


In [12]:
# Assuming you have already calculated the lower_threshold, upper_threshold, and extracted_df

# Iterate over the values and indices in the 'tokens' column
for index, value in extracted_df['tokens'].apply(len).items():
    if value < lower_threshold or value > upper_threshold:
        print("Outlier at index {}: {}".format(index, value))


Outlier at index 137: 682
Outlier at index 138: 672
Outlier at index 139: 662
Outlier at index 140: 653
Outlier at index 141: 647
Outlier at index 142: 639
Outlier at index 143: 636
Outlier at index 144: 595
Outlier at index 145: 585
Outlier at index 146: 574
Outlier at index 147: 555
Outlier at index 148: 543
Outlier at index 149: 536
Outlier at index 150: 534
Outlier at index 151: 518
Outlier at index 152: 515
Outlier at index 153: 496
Outlier at index 154: 495
Outlier at index 155: 472
Outlier at index 156: 462
Outlier at index 157: 449
Outlier at index 158: 447
Outlier at index 159: 432
Outlier at index 160: 427
Outlier at index 161: 414
Outlier at index 162: 403
Outlier at index 163: 398
Outlier at index 164: 383
Outlier at index 165: 374
Outlier at index 166: 367
Outlier at index 167: 356
Outlier at index 168: 354
Outlier at index 169: 316
Outlier at index 170: 303
Outlier at index 171: 301
Outlier at index 172: 236
Outlier at index 173: 225
Outlier at index 174: 211
Outlier at i

In [13]:
print(extracted_df.iloc[21]['url'])

https://www.amsterdam.nl/veelgevraagd/?categoryid=%7B317474FA-FF4D-43E1-BBA9-2C6C54A1E246%7D


In [14]:
# Assuming you have already calculated the lower_threshold, upper_threshold, and extracted_df

# Remove rows with empty 'text' column
extracted_df = extracted_df[extracted_df['text'] != '']

# Iterate over the values and indices in the updated 'tokens' column
for index, value in extracted_df['tokens'].apply(len).items():
    if value < lower_threshold or value > upper_threshold:
        print("Outlier at index {}: {}".format(index, value))


Outlier at index 137: 682
Outlier at index 138: 672
Outlier at index 139: 662
Outlier at index 140: 653
Outlier at index 141: 647
Outlier at index 142: 639
Outlier at index 143: 636
Outlier at index 144: 595
Outlier at index 145: 585
Outlier at index 146: 574
Outlier at index 147: 555
Outlier at index 148: 543
Outlier at index 149: 536
Outlier at index 150: 534
Outlier at index 151: 518
Outlier at index 152: 515
Outlier at index 153: 496
Outlier at index 154: 495
Outlier at index 155: 472
Outlier at index 156: 462
Outlier at index 157: 449
Outlier at index 158: 447
Outlier at index 159: 432
Outlier at index 160: 427
Outlier at index 161: 414
Outlier at index 162: 403
Outlier at index 163: 398
Outlier at index 164: 383
Outlier at index 165: 374
Outlier at index 166: 367
Outlier at index 167: 356
Outlier at index 168: 354
Outlier at index 169: 316
Outlier at index 170: 303
Outlier at index 171: 301
Outlier at index 172: 236
Outlier at index 173: 225
Outlier at index 174: 211
Outlier at i

In [15]:
# Assuming you have already calculated the lower_threshold, upper_threshold, and extracted_df

# Remove rows with empty 'text' column
extracted_df = extracted_df[extracted_df['text'] != '']

# Create an empty list to store the outliers
outliers = []

# Iterate over the values and indices in the updated 'tokens' column
for index, value in extracted_df['tokens'].apply(len).items():
    if value < lower_threshold or value > upper_threshold:
        outliers.append((index, value))

# Print the outliers
print("Outliers according to IQR:")
for index, value in outliers:
    print("Index: {}, Value: {}".format(index, value))


Outliers according to IQR:
Index: 137, Value: 682
Index: 138, Value: 672
Index: 139, Value: 662
Index: 140, Value: 653
Index: 141, Value: 647
Index: 142, Value: 639
Index: 143, Value: 636
Index: 144, Value: 595
Index: 145, Value: 585
Index: 146, Value: 574
Index: 147, Value: 555
Index: 148, Value: 543
Index: 149, Value: 536
Index: 150, Value: 534
Index: 151, Value: 518
Index: 152, Value: 515
Index: 153, Value: 496
Index: 154, Value: 495
Index: 155, Value: 472
Index: 156, Value: 462
Index: 157, Value: 449
Index: 158, Value: 447
Index: 159, Value: 432
Index: 160, Value: 427
Index: 161, Value: 414
Index: 162, Value: 403
Index: 163, Value: 398
Index: 164, Value: 383
Index: 165, Value: 374
Index: 166, Value: 367
Index: 167, Value: 356
Index: 168, Value: 354
Index: 169, Value: 316
Index: 170, Value: 303
Index: 171, Value: 301
Index: 172, Value: 236
Index: 173, Value: 225
Index: 174, Value: 211
Index: 175, Value: 210
Index: 195, Value: 553
Index: 196, Value: 189
Index: 210, Value: 1140
Index:

In [16]:
extracted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113089 entries, 0 to 121052
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     113089 non-null  object
 1   h1      113089 non-null  object
 2   h2      78483 non-null   object
 3   h3      45749 non-null   object
 4   h4      5295 non-null    object
 5   h5      0 non-null       object
 6   h6      0 non-null       object
 7   text    113089 non-null  object
 8   tokens  113089 non-null  object
dtypes: object(9)
memory usage: 8.6+ MB


In [17]:
# Remove rows with empty 'text' column
extracted_df = extracted_df[extracted_df['text'] != '']

# Calculate the mean number of tokens
mean_tokens = extracted_df['tokens'].apply(len).mean()

# Create a new DataFrame to store the split text chunks
split_df = pd.DataFrame(columns=['url', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'chunk'])

# Iterate over the rows in the updated DataFrame
for index, row in extracted_df.iterrows():
    tokens = row['tokens']
    h1 = row['h1']
    h2 = row['h2']
    h3 = row['h3']
    h4 = row['h4']
    h5 = row['h5']
    h6 = row['h6']
    text = row['text']
    url = row['url']

    if pd.isna(h1):
        h1 = None

    # Check if the 'h2' column contains NaN
    if pd.isna(h2):
        h2 = None

    # Check if the 'h3' column contains NaN
    if pd.isna(h3):
        h3 = None

    # Check if the 'h4' column contains NaN
    if pd.isna(h4):
        h4 = None
    
    # Check if the 'h3' column contains NaN
    if pd.isna(h5):
        h5 = None

    # Check if the 'h4' column contains NaN
    if pd.isna(h6):
        h6 = None
    


    if len(tokens) > upper_threshold:
        # Split the text into chunks
        chunks = []
        current_chunk = []
        for token in tokens:
            current_chunk.append(token)

            # Check if the current chunk length is close to the mean
            if abs(len(current_chunk) - mean_tokens) <= 2:
                chunks.append(' '.join(current_chunk))
                current_chunk = []

        # Create a DataFrame for the split chunks
        chunk_df = pd.DataFrame({'url': [url] * len(chunks),
                                 'h1': [h1] * len(chunks),
                                 'h2': [h2] * len(chunks),
                                 'h3': [h3] * len(chunks),
                                 'h4': [h4] * len(chunks),
                                 'h5': [h5] * len(chunks),
                                 'h6': [h6] * len(chunks),
                                 'chunk': chunks})
        
        # Append the split text chunks to the new DataFrame
        split_df = pd.concat([split_df, chunk_df], ignore_index=True)
    else:
        # If the number of tokens is not an outlier, keep the original text as a single chunk
        split_df = pd.concat([split_df, pd.DataFrame({'url': [url],
                                                      'h1': [h1],
                                                      'h2': [h2],
                                                      'h3': [h3],
                                                      'h4': [h4],
                                                      'h5': [h5],
                                                      'h6': [h6],
                                                      'chunk': [text]})],
                             ignore_index=True)
# 11:49 mins

In [18]:
split_df.head()

,url,h1,h2,h3,h4,h5,h6,chunk
0,https://www.amsterdam.nl/veelgevraagd/,Veelgevraagd,None,None,None,None,None,Burgerzaken\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThema...
1,https://www.amsterdam.nl/veelgevraagd/,Veelgevraagd,Thema's,None,None,None,None,Belasting en heffingen Burgerzaken Bestuur en ...
2,https://www.amsterdam.nl/veelgevraagd/,Veelgevraagd,Regel het online,None,None,None,None,Veel zaken kunt u 24/7 thuis online regelen. R...
3,https://www.amsterdam.nl/veelgevraagd/,Veelgevraagd,Uw gegevens,None,None,None,None,De status van uw aanvraag volgen? Zien hoe u g...
4,https://www.amsterdam.nl/veelgevraagd/,Gemeente Amsterdam,None,None,None,None,None,Contact\n\nHebt u een vraag en kunt u het antw...


In [19]:
# Calculate the average token length
avg_token_length = split_df['chunk'].apply(nltk.word_tokenize).apply(len).mean()

print("Average token length:", avg_token_length) # 1m:18 sec

Average token length: 56.126576772509786


In [20]:

# Tokenize the 'text' column
split_df['tokens'] = split_df['chunk'].apply(nltk.word_tokenize)

# Calculate the statistics
statistics = split_df['tokens'].apply(len).describe()

# Extract the desired statistics
median_tokens = statistics['50%']
min_tokens = statistics['min']
max_tokens = statistics['max']

# Print the statistics
print("Median number of tokens in 'text' column:", median_tokens)
print("Minimum number of tokens in 'text' column:", min_tokens)
print("Maximum number of tokens in 'text' column:", max_tokens)


Median number of tokens in 'text' column: 57.0
Minimum number of tokens in 'text' column: 1.0
Maximum number of tokens in 'text' column: 174.0


In [21]:
# Calculate the IQR and define the threshold for outliers
Q1 = split_df['tokens'].apply(len).quantile(0.25)
Q3 = split_df['tokens'].apply(len).quantile(0.75)
IQR = Q3 - Q1
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR

# Count the number of outliers
outliers_count = ((split_df['tokens'].apply(len) < lower_threshold) |
                  (split_df['tokens'].apply(len) > upper_threshold)).sum()

print(outliers_count)

# Iterate over the values and indices in the updated 'tokens' column
for index, value in split_df['tokens'].apply(len).items():
    if value < lower_threshold or value > upper_threshold:
        print("Outlier at index {}: {}".format(index, value))

969
Outlier at index 326: 163
Outlier at index 763: 166
Outlier at index 1361: 163
Outlier at index 1517: 163
Outlier at index 1656: 164
Outlier at index 1740: 173
Outlier at index 2226: 163
Outlier at index 2615: 170
Outlier at index 2648: 162
Outlier at index 2677: 167
Outlier at index 2722: 167
Outlier at index 3065: 170
Outlier at index 3066: 164
Outlier at index 3214: 171
Outlier at index 3510: 159
Outlier at index 3823: 168
Outlier at index 3824: 164
Outlier at index 3948: 164
Outlier at index 3950: 161
Outlier at index 4095: 173
Outlier at index 4325: 161
Outlier at index 4475: 167
Outlier at index 4610: 166
Outlier at index 4769: 161
Outlier at index 4800: 164
Outlier at index 4867: 163
Outlier at index 4894: 160
Outlier at index 4976: 171
Outlier at index 4977: 169
Outlier at index 5039: 173
Outlier at index 5485: 168
Outlier at index 5588: 166
Outlier at index 6558: 173
Outlier at index 6559: 162
Outlier at index 7240: 170
Outlier at index 7345: 172
Outlier at index 7382: 168

In [37]:
split_df.tail(20)

,url,h1,h2,h3,h4,h5,h6,chunk,tokens
156312,https://www.amsterdam.nl/nieuws/kennisgevingen...,Kennisgevingen en bekendmakingen,Op zoek naar een ouder besluit?,Besluit apv vergunning Verleend - Nepveustraat 12,None,None,None,05 juni 2023,"[05, juni, 2023]"
156313,https://www.amsterdam.nl/nieuws/kennisgevingen...,Kennisgevingen en bekendmakingen,Op zoek naar een ouder besluit?,Besluit apv vergunning Verleend - Wakkerstraat...,None,None,None,05 juni 2023,"[05, juni, 2023]"
156314,https://www.amsterdam.nl/nieuws/kennisgevingen...,Kennisgevingen en bekendmakingen,Op zoek naar een ouder besluit?,Besluit apv vergunning Verleend - Reinveen 10-H,None,None,None,05 juni 2023,"[05, juni, 2023]"
156315,https://www.amsterdam.nl/nieuws/kennisgevingen...,Kennisgevingen en bekendmakingen,Op zoek naar een ouder besluit?,Ingetrokken aanvraag omgevingsvergunning Rhône...,None,None,None,05 juni 2023,"[05, juni, 2023]"
156316,https://www.amsterdam.nl/nieuws/kennisgevingen...,Kennisgevingen en bekendmakingen,Op zoek naar een ouder besluit?,Besluit apv vergunning Verleend - Polluxplein 27,None,None,None,05 juni 2023,"[05, juni, 2023]"
156317,https://www.amsterdam.nl/nieuws/kennisgevingen...,Gemeente Amsterdam,None,None,None,None,None,Contact\n\nHebt u een vraag en kunt u het antw...,"[Contact, Hebt, u, een, vraag, en, kunt, u, he..."
156318,https://www.amsterdam.nl/nieuws/kennisgevingen...,Gemeente Amsterdam,Contact,None,None,None,None,Hebt u een vraag en kunt u het antwoord niet v...,"[Hebt, u, een, vraag, en, kunt, u, het, antwoo..."
156319,https://www.amsterdam.nl/nieuws/kennisgevingen...,Gemeente Amsterdam,Volg de gemeente,None,None,None,None,Nieuwsbrief Amsterdam\nTwitter\nFacebook\nInst...,"[Nieuwsbrief, Amsterdam, Twitter, Facebook, In..."
156320,https://www.amsterdam.nl/nieuws/kennisgevingen...,Gemeente Amsterdam,Kalender,None,None,None,None,Van buurtactiviteiten tot inspraakavonden. Wat...,"[Van, buurtactiviteiten, tot, inspraakavonden,..."
156321,https://www.amsterdam.nl/nieuws/kennisgevingen...,Gemeente Amsterdam,Kalender,Uit in Amsterdam,None,None,None,Benieuwd wat er allemaal te doen is in de stad...,"[Benieuwd, wat, er, allemaal, te, doen, is, in..."


In [ ]:
# to drop:
# Volg de gemeente
# Contact\n\nHebt u een vraag en kunt u het antwoord niet vinden op deze website? Neem dan contact met ons op.\n\nContactformulier\nBel het telefoonnummer 14 020maandag tot en met vrijdag van 08.00 tot 18.00 uur\nContactgegevens en openingstijden\n\n\n\n\n\n\n\n\n\n\nVolg de gemeente\n\nNieuwsbrief Amsterdam\nTwitter\nFacebook\nInstagram\nLinkedIn\nYouTube\nWerkenbij\n\n\n\n\n\n\n\n\n\nKalender\n\nVan buurtactiviteiten tot inspraakavonden. Wat organiseert de gemeente voor u?\xa0Kijk op Kalender Amsterdam\nUit in Amsterdam\nBenieuwd wat er allemaal te doen is in de stad? Op\xa0Iamsterdam.com\xa0vindt u de beste tips op het gebied van cultuur, uitgaan en evenementen. Lijst\n\nOver deze site\nPrivacy\nCookies op deze site\nWebarchief
# Over deze site\nPrivacy\nCookies op deze site\nWebarchief
# Contact
# Lijst

In [38]:
mask = split_df['h2'].isin(['Contact', 'Lijst', 'Volg de gemeente'])
split_df = split_df[~mask]
split_df = split_df.drop(split_df[mask].index)

<ipython-input-38-3d182eedb258>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  split_df = split_df.drop(split_df[mask].index)


In [41]:
mask = split_df['chunk'].isin(['Contact\n\nHebt u een vraag en kunt u het antwoord niet vinden op deze website? Neem dan contact met ons op.\n\nContactformulier\nBel het telefoonnummer 14 020maandag tot en met vrijdag van 08.00 tot 18.00 uur\nContactgegevens en openingstijden\n\n\n\n\n\n\n\n\n\n\nVolg de gemeente\n\nNieuwsbrief Amsterdam\nTwitter\nFacebook\nInstagram\nLinkedIn\nYouTube\nWerkenbij\n\n\n\n\n\n\n\n\n\nKalender\n\nVan buurtactiviteiten tot inspraakavonden. Wat organiseert de gemeente voor u?\xa0Kijk op Kalender Amsterdam\nUit in Amsterdam\nBenieuwd wat er allemaal te doen is in de stad? Op\xa0Iamsterdam.com\xa0vindt u de beste tips op het gebied van cultuur, uitgaan en evenementen. Lijst\n\nOver deze site\nPrivacy\nCookies op deze site\nWebarchief', 'Over deze site\nPrivacy\nCookies op deze site\nWebarchief'])
split_df = split_df[~mask]
split_df = split_df.drop(split_df[mask].index)

<ipython-input-41-3120f0d10aff>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  split_df = split_df.drop(split_df[mask].index)


In [44]:
# Define a lambda function to concatenate the desired columns
concat_func = lambda row: ' '.join(filter(lambda x: x is not None, [row['h1'], row['h2'], row['h3'], row['h5'], row['h6'], row['chunk']]))

# Create a new DataFrame 'new_df' with columns: URL, Textual_Content
new_df = pd.DataFrame({'URL': split_df['url'], 'Textual_Content': split_df.apply(concat_func, axis=1)})

In [48]:
len(new_df)

120907

In [43]:
split_df.to_csv('data/collected/paragraphs_dropped_junk.csv')

In [47]:
new_df.to_csv('data/collected/paragraphs_dropped_junk_concatenated.csv')